In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score,precision_score,recall_score,f1_score,roc_auc_score

In [2]:
df = pd.read_csv('../data/crawler/unified-events-statistics.csv')
df = df.replace(['BLUE: first_blood',
    'BLUE: dragon',
    'BLUE: herald',
    'BLUE: first_tower_top',
    'BLUE: first_tower_mid',
    'BLUE: first_tower_bot',
    'BLUE: second_tower_top',
    'BLUE: second_tower_mid',
    'BLUE: second_tower_bot',
    'BLUE: third_tower_top',
    'BLUE: third_tower_mid',
    'BLUE: third_tower_bot',
    'BLUE: inhibitor_top',
    'BLUE: inhibitor_mid',
    'BLUE: inhibitor_bot',
    'BLUE: baron',
    'BLUE: elder_dragon',
    'BLUE: nexus_tower',
    'BLUE: nexus',
    'RED: first_blood',
    'RED: dragon',
    'RED: herald',
    'RED: first_tower_top',
    'RED: first_tower_mid',
    'RED: first_tower_bot',
    'RED: second_tower_top',
    'RED: second_tower_mid',
    'RED: second_tower_bot',
    'RED: third_tower_top',
    'RED: third_tower_mid',
    'RED: third_tower_bot',
    'RED: inhibitor_top',
    'RED: inhibitor_mid',
    'RED: inhibitor_bot',
    'RED: baron',
    'RED: elder_dragon',
    'RED: nexus_tower',
    'RED: nexus'], range(38))
df.head()

,golId,game,event1,event2,event3,event4,event5,event6,event7,event8,...,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA,result
0,36864,ESPORTSTMNT02_2556988,19,1,2,1,3,24,2,23,...,5,0.20,4.00,0,0.0,0.00,0,0.0,0.0,1
1,36865,ESPORTSTMNT05_2520933,19,21,1,22,25,24,20,21,...,4,0.25,3.55,10,0.4,4.07,2,0.5,5.4,0
2,36866,ESPORTSTMNT02_2557390,19,2,20,24,2,4,20,3,...,1,0.00,20.00,0,0.0,0.00,4,1.0,2.8,0
3,36867,ESPORTSTMNT02_2557426,20,0,21,20,21,5,23,1,...,5,0.20,4.00,5,0.6,9.12,0,0.0,0.0,0
4,36868,ESPORTSTMNT02_2557463,19,1,2,1,2,4,20,24,...,5,0.00,2.35,2,0.0,24.00,0,0.0,0.0,0


In [3]:
y = df['result'].copy()
X = df.drop(['golId','result','game','event6','event7','event8','event9','event10','event11','event12','event13','event14','event15','event16','event17','event18','event19','event20','event21','event22','event23','event24','event25','event26','event27','event28','event29','event30','event31','event32','event33','event34','event35','event36','event37','event38','event39','event40','event41','event42','event43','event44','event45'],axis=1)
X.head()

,event1,event2,event3,event4,event5,blueTopGP,blueTopWR,blueTopKDA,blueJungleGP,blueJungleWR,...,redJungleKDA,redMidGP,redMidWR,redMidKDA,redAdcGP,redAdcWR,redAdcKDA,redSupportGP,redSupportWR,redSupportKDA
0,19,1,2,1,3,1,1.00,2.00,1,1.0,...,0.0,5,0.20,4.00,0,0.0,0.00,0,0.0,0.0
1,19,21,1,22,25,1,0.00,3.00,2,1.0,...,4.0,4,0.25,3.55,10,0.4,4.07,2,0.5,5.4
2,19,2,20,24,2,0,0.00,0.00,1,0.0,...,0.0,1,0.00,20.00,0,0.0,0.00,4,1.0,2.8
3,20,0,21,20,21,7,0.14,4.14,1,0.0,...,0.0,5,0.20,4.00,5,0.6,9.12,0,0.0,0.0
4,19,1,2,1,2,1,0.00,2.00,1,1.0,...,6.6,5,0.00,2.35,2,0.0,24.00,0,0.0,0.0


In [4]:
def preprocess_input(X,y):
    X = X.copy()
    X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,stratify=y)
    scaler = StandardScaler()   
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train,X_test,y_train,y_test,scaler

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

executions = pd.DataFrame()

for i in range(50):
  print('execução número ' + str(i))
  X_train,X_test,y_train,y_test,scaler = preprocess_input(X,y)

  models = {
      'Logistic Regression': LogisticRegression(max_iter=10000),
      'Support Vector Machine (RBF Kernel)': SVC(C=1,gamma=0.01,kernel='sigmoid',max_iter=50000,probability=True),
      'Decission Tree': DecisionTreeClassifier(),
      'Adaboost': AdaBoostClassifier(),
      'Random Forest': RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=5,min_samples_leaf=1),
      'Gradient Boosting Classifier': GradientBoostingClassifier(),
      'KNN': KNeighborsClassifier(n_neighbors=3),
      'Gaussian NB': GaussianNB()
  }

  for name, model in models.items():
    model.fit(X_train,y_train)

  scores_list = []

  for name,model in models.items():    
      scores_list.append({
      'model': name,
      'amount_events': '5',
      'execution': i,
      'Balanced Accuracy': balanced_accuracy_score(y_test,model.predict(X_test)),
      'Precision':  precision_score(y_test,model.predict(X_test)),
      'Recall': recall_score(y_test,model.predict(X_test)),
      'F1-Score': f1_score(y_test,model.predict(X_test)),
      'auc': roc_auc_score(y_test,model.predict_proba(X_test)[:,1])
      })
  # scores = pd.DataFrame(scores_list)
  executions = executions.append(scores_list)

execução número 0
execução número 1
execução número 2
execução número 3
execução número 4
execução número 5
execução número 6
execução número 7
execução número 8
execução número 9
execução número 10
execução número 11
execução número 12
execução número 13
execução número 14
execução número 15
execução número 16
execução número 17
execução número 18
execução número 19
execução número 20
execução número 21
execução número 22
execução número 23
execução número 24
execução número 25
execução número 26
execução número 27
execução número 28
execução número 29
execução número 30
execução número 31
execução número 32
execução número 33
execução número 34
execução número 35
execução número 36
execução número 37
execução número 38
execução número 39
execução número 40
execução número 41
execução número 42
execução número 43
execução número 44
execução número 45
execução número 46
execução número 47
execução número 48
execução número 49


In [6]:
avg_scores_list = []

for name, model in models.items():

    avg_balanced_acc = 0

    model_metrics = executions.loc[executions['model'] == name]
    avg_balanced_acc = model_metrics['Balanced Accuracy'].sum() / len(model_metrics['Balanced Accuracy'])
    avg_precision = model_metrics['Precision'].sum() / len(model_metrics['Precision'])
    avg_recall = model_metrics['Recall'].sum() / len(model_metrics['Recall'])
    avg_f_score = model_metrics['F1-Score'].sum() / len(model_metrics['F1-Score'])
    avg_auc = model_metrics['auc'].sum() / len(model_metrics['auc'])

    avg_scores_list.append({
      'model': name,
      'amount_events': '5',
      'Balanced Accuracy': avg_balanced_acc,
      'Precision': avg_precision,
      'Recall': avg_recall,
      'F1-Score': avg_f_score,
      'auc': avg_auc
      })
avg_scores = pd.DataFrame(avg_scores_list)

In [7]:
best_model = avg_scores.sort_values(by='auc', ascending=False).iloc[0]['model']
avg_scores.sort_values(by='auc', ascending=False)

,model,amount_events,Balanced Accuracy,Precision,Recall,F1-Score,auc
4,Random Forest,5,0.701535,0.699993,0.731053,0.715014,0.772543
0,Logistic Regression,5,0.698387,0.699656,0.720307,0.709703,0.769266
1,Support Vector Machine (RBF Kernel),5,0.698831,0.696459,0.730965,0.713184,0.769116
5,Gradient Boosting Classifier,5,0.694207,0.697074,0.712588,0.704537,0.766342
3,Adaboost,5,0.693214,0.696000,0.711886,0.703711,0.763448
7,Gaussian NB,5,0.676393,0.679665,0.696272,0.687702,0.731814
6,KNN,5,0.620647,0.628220,0.633772,0.630871,0.648333
2,Decission Tree,5,0.601803,0.610900,0.610395,0.610441,0.601803


In [8]:
header = ['model', 'amount_events', 'auc']
avg_scores.to_csv('../data/models-analysis/models.csv', columns = header, mode='a',index=False,header=False)

In [9]:
import pickle

with open("../models/model-5.pkl", "wb") as f:
    pickle.dump(models[best_model], f)

with open("../scalers/scaler-5.pkl", "wb") as f:
    pickle.dump(scaler, f)